In [1]:
# %pip install -q causal-learn

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

from sklearn.preprocessing import StandardScaler
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib

# causal-learn: 정확한 GES 구현
from causallearn.search.ScoreBased.GES import ges   # 기본 점수는 BIC
# from causallearn.score.LocalScoreFunction import local_score_BIC

# 한글 폰트/마이너스 설정(선택)
plt.rcParams['font.family'] = 'Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False

# 0) 데이터 로드 (기존과 동일)
df = pd.read_csv("training_data.csv")
feature_cols = df.drop(columns=["Unnamed: 0", "label"]).columns
X_base = df[feature_cols].copy()
y = df["label"].astype(int)
cols_for_w = list(feature_cols)
cols = cols_for_w  # 시각화 라벨 호환

# 1) 정확한 GES 실행 (기본 BIC 점수 사용)
X_np = X_base.values.astype(float)
Record = ges(X_np)  # 또는 Record = ges(X_np, score_func='local_score_BIC')
Gmat = Record['G'].graph  # orientation 행렬 (numpy.ndarray)
# 문서: G.graph[j,i]=1 & G.graph[i,j]=-1 이면 i -> j (i가 부모, j가 자식)

# 2) 방향성 인접행렬 A (i <- j 형태로 저장하기 위해 A[child, parent]=1)
p = len(cols_for_w)
A = np.zeros((p, p), dtype=float)
for i in range(p):
    for j in range(p):
        if i == j:
            continue
        # i -> j ?
        if (Gmat[j, i] == 1) and (Gmat[i, j] == -1):
            A[j, i] = 1.0  # child=j, parent=i

# 3) 표준화 OLS로 엣지 가중치 행렬 W (i <- j)
Z = StandardScaler().fit_transform(X_np)
W = np.zeros_like(A, dtype=float)
for child in range(p):
    parents = np.where(A[child, :] == 1.0)[0]
    if parents.size == 0:
        continue
    y_child = Z[:, child]
    X_par = Z[:, parents]
    coef, *_ = np.linalg.lstsq(X_par, y_child, rcond=None)  # 절편 없음(표준화)
    W[child, parents] = coef.astype(float)

# ============ 여기부터 시각화 개선(음수 가중치 지원) ============

# 시각화 임계 설정
USE_QUANTILE = False      # 분위수 임계 사용 여부
W_THRESHOLD = 0.01        # 고정 임계(|W|>0.01)
W_ABS_QUANTILE = 0.90     # 상위 10%만 (USE_QUANTILE=True일 때 사용)

TOP_K_EDGES = None        # 상위 |w| 간선만 표시하고 싶으면 정수(예: 100)로 설정

def get_threshold(W, fixed=W_THRESHOLD, use_q=USE_QUANTILE, q=W_ABS_QUANTILE):
    if not use_q:
        return fixed
    aw = np.abs(W).ravel()
    aw = aw[aw > 0]
    return float(np.quantile(aw, q)) if aw.size else fixed

thr = get_threshold(W)

# 4) 임계 적용 + (선택) 상위 K개 필터 → DiGraph 구성
edges_all = []
for i in range(p):
    for j in range(p):
        if i == j:
            continue
        w = float(W[i, j])
        if abs(w) > thr:
            edges_all.append((cols[j], cols[i], w))  # parent -> child, weight=w

# 상위 K개만 표시(선택)
if TOP_K_EDGES is not None and TOP_K_EDGES > 0 and len(edges_all) > TOP_K_EDGES:
    edges_all = sorted(edges_all, key=lambda t: abs(t[2]), reverse=True)[:TOP_K_EDGES]

G = nx.DiGraph()
for u, v, w in edges_all:
    G.add_edge(u, v, weight=round(w, 3))

# ---- 배치: 음수 가중치 문제 해결 (거리=1/(|w|+ε)로 양의 거리 그래프 H 구성)
eps = 1e-6
H = nx.Graph()  # 레이아웃 전용 보조 그래프(무향 + 양의 거리)
for u, v, d in G.edges(data=True):
    w = d.get("weight", 0.0)
    dist = 1.0 / (abs(w) + eps)   # |w| 클수록 더 가까워지도록
    H.add_edge(u, v, weight=dist)

pos = nx.kamada_kawai_layout(H, weight='weight')  # 안정적 배치

# ---- 스타일: 부호/크기 시각화
max_abs = max((abs(d["weight"]) for _, _, d in G.edges(data=True)), default=1.0)
pos_edges = [(u, v) for u, v, d in G.edges(data=True) if d["weight"] >= 0]
neg_edges = [(u, v) for u, v, d in G.edges(data=True) if d["weight"] < 0]
pos_widths = [1.0 + 3.0 * abs(G[u][v]["weight"]) / max_abs for u, v in pos_edges]
neg_widths = [1.0 + 3.0 * abs(G[u][v]["weight"]) / max_abs for u, v in neg_edges]

plt.figure(figsize=(10, 8))
nx.draw_networkx_nodes(G, pos, node_size=2000, node_color="white", edgecolors="black")
nx.draw_networkx_labels(G, pos, font_size=10)

# 양수: 파란 실선 / 음수: 빨간 점선
nx.draw_networkx_edges(G, pos, edgelist=pos_edges, edge_color="tab:blue", width=pos_widths, arrows=True)
nx.draw_networkx_edges(G, pos, edgelist=neg_edges, edge_color="tab:red",  width=neg_widths, style="dashed", arrows=True)

# 원래 가중치 라벨(부호 유지)
nx.draw_networkx_edge_labels(G, pos, edge_labels=nx.get_edge_attributes(G, 'weight'), font_size=8)
plt.title("정확한 GES 기반 인과 그래프 (배치=|w| 거리, 색/스타일=부호, 굵기=|w|)")
plt.axis("off")
plt.show()


가중치 무작위

In [ ]:
# ============================================
# Random Edges → Top-K → mul/mulw → LightGBM → O/F/OF
# 10-fold CV, O는 1회만(값 재사용) → 총 50회 학습
# 결과: 행[original, m, mw] × 열[O,F,OF × f1,AUPRC,AUROC,Brier,ECE]
# Python 3.9.1 / LightGBM 3.x~4.x 호환 (callback 기반 early stopping)
# ============================================

from pathlib import Path
from typing import List, Dict
import numpy as np
import pandas as pd
import warnings

# 경고 최소화
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(
    "ignore",
    message="X does not have valid feature names, but LGBMClassifier was fitted with feature names"
)
warnings.filterwarnings("ignore", category=DataConversionWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, brier_score_loss
from lightgbm import LGBMClassifier, early_stopping, log_evaluation

# ---------------- 설정 ----------------
DATA_PATH = "./training_data.csv"   # 입력 파일
LABEL_COL = "label"
ID_PREFIX = "Unnamed"               # 식별자 접두사(예: 'Unnamed: 0')

TOP_K_EDGES   = 15                  # 무작위 엣지(유향, child<-parent) 개수
N_SPLITS      = 10
RANDOM_STATE  = 42
PRINT_PROGRESS = False              # True로 바꾸면 fold 1·마지막만 간단 출력

# LightGBM 파라미터(안정/경량 기본값)
LGBM_PARAMS = dict(
    n_estimators=300,
    learning_rate=0.05,
    num_leaves=63,
    max_depth=-1,                # 자동
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    reg_alpha=0.0,
    objective="binary",
    class_weight="balanced",     # 불균형 자동 보정
    random_state=RANDOM_STATE,
    n_jobs=-1,
    verbosity=-1,                # 내부 로그 비활성화
)
EARLY_STOPPING_ROUNDS = 50
# -------------------------------------


# 1) 데이터 로드 & 컬럼 준비
path = Path(DATA_PATH)
assert path.exists(), f"파일을 찾을 수 없습니다: {path}"
df = pd.read_csv(path)

assert LABEL_COL in df.columns, f"'{LABEL_COL}' 컬럼이 없습니다."
id_cols = [c for c in df.columns if c.startswith(ID_PREFIX)]
feature_cols = [c for c in df.columns if c not in id_cols + [LABEL_COL]]

X_base = df[feature_cols].copy()
y = df[LABEL_COL].astype(int).copy()

# 상수(분산 0) 피처 제거 → 실사용 피처 목록
var0_cols = [c for c in feature_cols if np.isclose(X_base[c].var(ddof=0), 0.0)]
cols_used = [c for c in feature_cols if c not in var0_cols]
p = len(cols_used)
assert p >= 2, "무작위 엣지를 만들려면 최소 2개 이상의 피처가 필요합니다."

# 2) 무작위 엣지 및 가중치 생성 (child <- parent)
rng = np.random.default_rng(RANDOM_STATE)
all_pairs = [(child, parent) for child in range(p) for parent in range(p) if child != parent]
max_edges = len(all_pairs)
k = min(TOP_K_EDGES, max_edges)
chosen_idx = rng.choice(max_edges, size=k, replace=False)
chosen_edges = [all_pairs[idx] for idx in chosen_idx]
weights = rng.uniform(-1.0, 1.0, size=k)   # U(-1,1)

# 방향성 가중치 행렬 W_used (i <- j, child=i, parent=j)
W_used = np.zeros((p, p), dtype=float)
for (child, parent), w in zip(chosen_edges, weights):
    W_used[child, parent] = w


# === 유틸 ===
def pick_top_k_edges(W: np.ndarray, cols: List[str], top_k: int = 15):
    """
    (i <- j, weight=W[i,j]) 기준의 상위 |w| 간선 선택
    반환: [(i, j, w_ij), ...] with i:child, j:parent
    """
    n = W.shape[0]
    flat = []
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            w = W[i, j]
            if w == 0.0:
                continue
            flat.append((i, j, w))
    if not flat:
        return []
    flat.sort(key=lambda t: (abs(t[2]), t[2]), reverse=True)
    return flat[:min(top_k, len(flat))]


def expected_calibration_error(y_true, y_prob, n_bins: int = 10):
    y_true = np.asarray(y_true)
    y_prob = np.asarray(y_prob)
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    ece = 0.0
    total = len(y_true)
    for b in range(n_bins):
        left, right = bins[b], bins[b+1]
        if b < n_bins - 1:
            mask = (y_prob >= left) & (y_prob < right)
        else:
            mask = (y_prob >= left) & (y_prob <= right)
        if not np.any(mask):
            continue
        acc = y_true[mask].mean()
        conf = y_prob[mask].mean()
        ece += (mask.sum() / total) * abs(acc - conf)
    return float(ece)


def build_feature_df(
    X_base_df: pd.DataFrame,
    cols: List[str],
    W: np.ndarray,
    feature_type: str = "mul",
    top_k: int = 15
) -> pd.DataFrame:
    """
    feature_type in {"mul","mulw"}:
      - mul  : (A * B)
      - mulw : w_ij * (A * B)
    규약: w_ij = W[i,j], A=cols[j](from/parent), B=cols[i](to/child)
    """
    edges = pick_top_k_edges(W, cols, top_k=top_k)
    if len(edges) == 0:
        # 엣지가 하나도 없으면 빈 DF 반환
        return pd.DataFrame(index=X_base_df.index)

    feats = {}
    for i, j, w_ij in edges:
        A_name = cols[j]; B_name = cols[i]
        if feature_type == "mul":
            name = f"{A_name}_mul_{B_name}"
            val = X_base_df[A_name] * X_base_df[B_name]
        elif feature_type == "mulw":
            name = f"{A_name}_mulw_{B_name}"
            val = w_ij * (X_base_df[A_name] * X_base_df[B_name])
        else:
            raise ValueError("feature_type must be 'mul' or 'mulw'")
        feats[name] = val.values
    return pd.DataFrame(feats, index=X_base_df.index)


# === LightGBM 교차검증 ===
def run_cv_lgbm(X_df: pd.DataFrame, y_sr: pd.Series, n_splits=10, random_state=42, tag: str = "") -> Dict[str, float]:
    # pandas 타입 유지: 경고 방지 + 일관성
    assert isinstance(X_df, pd.DataFrame) and isinstance(y_sr, (pd.Series, pd.DataFrame)), "Use pandas types"

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    f1s, auprcs, aurocs, briers, eces = [], [], [], [], []

    for fold, (tr_idx, te_idx) in enumerate(skf.split(X_df, y_sr), start=1):
        if PRINT_PROGRESS and tag and fold in (1, n_splits):
            print(f"[{tag}] fold {fold}/{n_splits} ...", flush=True)

        X_tr, X_te = X_df.iloc[tr_idx], X_df.iloc[te_idx]
        y_tr, y_te = y_sr.iloc[tr_idx], y_sr.iloc[te_idx]

        # 내부 검증(조기 종료)
        X_tr_sub, X_val, y_tr_sub, y_val = train_test_split(
            X_tr, y_tr, test_size=0.2, random_state=random_state, stratify=y_tr
        )

        clf = LGBMClassifier(**LGBM_PARAMS)
        clf.fit(
            X_tr_sub, y_tr_sub,
            eval_set=[(X_val, y_val)],
            eval_metric="auc",
            callbacks=[
                early_stopping(EARLY_STOPPING_ROUNDS, verbose=False),  # 조기종료 메시지 끔
                log_evaluation(0),                                    # 평가 로그 끔
            ],
        )

        y_proba = clf.predict_proba(X_te)[:, 1]
        y_pred  = (y_proba >= 0.5).astype(int)

        f1s.append(f1_score(y_te, y_pred, zero_division=0))
        try:
            aurocs.append(roc_auc_score(y_te, y_proba))
        except Exception:
            aurocs.append(np.nan)
        try:
            auprcs.append(average_precision_score(y_te, y_proba))
        except Exception:
            auprcs.append(np.nan)
        briers.append(brier_score_loss(y_te, y_proba))
        eces.append(expected_calibration_error(y_te, y_proba, n_bins=10))

    return {
        "f1": float(np.nanmean(f1s)),
        "AUPRC": float(np.nanmean(auprcs)),
        "AUROC": float(np.nanmean(aurocs)),
        "Brier": float(np.nanmean(briers)),
        "ECE": float(np.nanmean(eces)),
    }


# 3) 파생 피처 생성 (무작위 W 기반)
X_used_df = X_base[cols_used]               # 파생은 cols_used만 사용
Xf_m   = build_feature_df(X_used_df, cols_used, W_used, feature_type="mul",  top_k=TOP_K_EDGES)
Xf_mw  = build_feature_df(X_used_df, cols_used, W_used, feature_type="mulw", top_k=TOP_K_EDGES)

# 4) 평가 (O는 1회만 학습 → 재사용)
res_O      = run_cv_lgbm(X_base, y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag="O(original)")

res_F_m    = run_cv_lgbm(Xf_m,  y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag="F(m)")
res_OF_m   = run_cv_lgbm(pd.concat([X_base, Xf_m], axis=1),  y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag="OF(m)")

res_F_mw   = run_cv_lgbm(Xf_mw, y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag="F(mw)")
res_OF_mw  = run_cv_lgbm(pd.concat([X_base, Xf_mw], axis=1), y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag="OF(mw)")

# 5) 결과 표 구성 (MultiIndex 컬럼, 변수명 report)
col_top = ["O", "F", "OF"]
col_metrics = ["f1", "AUPRC", "AUROC", "Brier", "ECE"]
multi_cols = pd.MultiIndex.from_product([col_top, col_metrics])
index_rows = ["original", "m", "mw"]  # m+mw 제거

report = pd.DataFrame(index=index_rows, columns=multi_cols, dtype=float)

# original 행: O만
for met in col_metrics:
    report.loc["original", ("O", met)] = res_O[met]

# m / mw 행
def put(row, res_F, res_OF):
    for met in col_metrics:
        report.loc[row, ("O", met)]  = res_O[met]   # O는 재사용(미재학습)
        report.loc[row, ("F", met)]  = res_F[met]
        report.loc[row, ("OF", met)] = res_OF[met]

put("m",   res_F_m,  res_OF_m)
put("mw",  res_F_mw, res_OF_mw)

report = report.astype(float).round(4)
report

[O(original)] fold 1/10 ...
[LightGBM] [Info] Number of positive: 285, number of negative: 12588
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000822 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3030
[LightGBM] [Info] Number of data points in the train set: 12873, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[22]	valid_0's auc: 0.928258	valid_0's binary_logloss: 0.286746
[O(original)] fold 2/10 ...
[LightGBM] [Info] Number of positive: 286, number of negative: 12588
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000356 seconds.
You can set `force_col_wise=true` to remove the overhead.

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Early stopping, best iteration is:
[106]	valid_0's auc: 0.920933	valid_0's binary_logloss: 0.0914904
[O(original)] fold 3/10 ...
[LightGBM] [Info] Number of positive: 286, number of negative: 12588
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000384 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3025
[LightGBM] [Info] Number of data points in the train set: 12874, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Training until validation scores don't improve for 50 rounds


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Early stopping, best iteration is:
[82]	valid_0's auc: 0.953858	valid_0's binary_logloss: 0.105681
[O(original)] fold 4/10 ...
[LightGBM] [Info] Number of positive: 286, number of negative: 12588
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000528 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3034
[LightGBM] [Info] Number of data points in the train set: 12874, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Training until validation scores don't improve for 50 rounds


...

Early stopping, best iteration is:
[108]	valid_0's auc: 0.947707	valid_0's binary_logloss: 0.0742498
[OF(m+mw)] fold 10/10 ...
[LightGBM] [Info] Number of positive: 285, number of negative: 12589
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001468 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10683
[LightGBM] [Info] Number of data points in the train set: 12874, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Training until validation scores don't improve for 50 rounds


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Early stopping, best iteration is:
[63]	valid_0's auc: 0.929991	valid_0's binary_logloss: 0.116287


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


O                                      F                  \
              f1   AUPRC   AUROC  Brier     ECE      f1   AUPRC   AUROC   
original  0.3923  0.4337  0.9152  0.042  0.0952     NaN     NaN     NaN   
m         0.3923  0.4337  0.9152  0.042  0.0952  0.4296  0.4725  0.9119   
mw        0.3923  0.4337  0.9152  0.042  0.0952  0.4602  0.4915  0.9129   
m+mw      0.3923  0.4337  0.9152  0.042  0.0952  0.4381  0.4750  0.9125   

                              OF                                  
           Brier     ECE      f1   AUPRC   AUROC   Brier     ECE  
original     NaN     NaN     NaN     NaN     NaN     NaN     NaN  
m         0.0294  0.0564  0.4581  0.5015  0.9196  0.0285  0.0550  
mw        0.0230  0.0301  0.4535  0.5039  0.9202  0.0324  0.0716  
m+mw      0.0238  0.0299  0.4198  0.4681  0.9113  0.0454  0.1050

In [ ]:
# report: MultiIndex 컬럼의 결과 DataFrame이 이미 존재한다고 가정
# format 옵션: "multi" | "flat" | "long"
save_format = "multi"  # 원하는 포맷으로 변경

if save_format == "multi":
    out_path = "report_ges_lightgbm_rand_w.csv"
    report.to_csv(out_path, encoding="utf-8-sig")

elif save_format == "flat":
    out_path = "report_ges_lightgbm_rand_w.csv"
    flat_cols = [f"{top}_{met}" for top, met in report.columns.to_list()]
    report_flat = report.copy()
    report_flat.columns = flat_cols
    report_flat.to_csv(out_path, encoding="utf-8-sig")

elif save_format == "long":
    out_path = "report_ges_lightgbm_rand_w.csv"
    report_long = (
        report
        .rename_axis(index="row", columns=["set", "metric"])
        .stack(["set", "metric"])
        .to_frame("value")
        .reset_index()[["row", "set", "metric", "value"]]
    )
    report_long.to_csv(out_path, index=False, encoding="utf-8-sig")
else:
    raise ValueError("save_format must be one of: 'multi', 'flat', 'long'")

print(f"Saved: {out_path} (format={save_format})")


Saved: report_ges_lightgbm_rand_w.csv (format=multi)


엣지 연결 무작위

In [ ]:
# ============================================
# Random DAG → Top-15 → mul/mulw → LightGBM → O/F/OF
# 10-fold CV, O는 1회만 → 총 70회 학습
# 결과: 행[original, m, mw, m+mw] × 열[O,F,OF × f1,AUPRC,AUROC,Brier,ECE]
# ============================================

import numpy as np
import pandas as pd
from pathlib import Path
from typing import List, Dict

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, brier_score_loss
from sklearn.preprocessing import StandardScaler

# LightGBM
from lightgbm import LGBMClassifier, early_stopping, log_evaluation

# ---------------- 설정 ----------------
DATA_PATH = "./training_data.csv"   # 입력 파일
LABEL_COL = "label"
ID_PREFIX = "Unnamed"               # 식별자 접두사(예: 'Unnamed: 0')

TOP_K_EDGES   = 15
N_SPLITS      = 10
RANDOM_STATE  = 42
PRINT_PROGRESS = True

LGBM_PARAMS = dict(
    n_estimators=300,
    learning_rate=0.05,
    num_leaves=63,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    reg_alpha=0.0,
    objective="binary",
    class_weight="balanced",
    random_state=RANDOM_STATE,
    n_jobs=-1
)
EARLY_STOPPING_ROUNDS = 50
# -------------------------------------


# 1) 데이터 로드
path = Path(DATA_PATH)
assert path.exists(), f"파일을 찾을 수 없습니다: {path}"
df = pd.read_csv(path)

assert LABEL_COL in df.columns, f"'{LABEL_COL}' 컬럼이 없습니다."
id_cols = [c for c in df.columns if c.startswith(ID_PREFIX)]
feature_cols = [c for c in df.columns if c not in id_cols + [LABEL_COL]]

X_base = df[feature_cols].copy()
y = df[LABEL_COL].astype(int).copy()

# 표준화 (랜덤 W 실험 시 scale 통일용)
Z = StandardScaler().fit_transform(X_base.values)
cols_used = feature_cols
p = len(cols_used)

# 2) 랜덤 DAG + 랜덤 가중치 생성
rng = np.random.default_rng(RANDOM_STATE)

# 무작위 방향 그래프 (상삼각만 허용하여 DAG 보장)
A = np.zeros((p, p), dtype=float)
for i in range(p):
    for j in range(i+1, p):  # i < j → i→j 혹은 j→i 중 하나 가능
        if rng.random() < 0.2:   # 20% 확률로 간선 생성
            if rng.random() < 0.5:
                A[j, i] = 1.0   # i → j
            else:
                A[i, j] = 1.0   # j → i

# 랜덤 가중치 행렬 (연결된 edge만)
W_used = np.zeros_like(A, dtype=float)
for child in range(p):
    parents = np.where(A[child, :] == 1.0)[0]
    if parents.size == 0:
        continue
    rand_weights = rng.uniform(-1.0, 1.0, size=parents.size)
    W_used[child, parents] = rand_weights


# === 유틸 ===
def pick_top_k_edges(W: np.ndarray, cols: List[str], top_k: int = 15):
    n = W.shape[0]
    flat = []
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            w = W[i, j]
            if w == 0.0:
                continue
            flat.append((i, j, w))
    flat.sort(key=lambda t: (abs(t[2]), t[2]), reverse=True)
    return flat[:top_k]

def expected_calibration_error(y_true, y_prob, n_bins: int = 10):
    y_true = np.asarray(y_true)
    y_prob = np.asarray(y_prob)
    bins = np.linspace(0.0, 1.0, n_bins + 1)
    ece = 0.0
    total = len(y_true)
    for b in range(n_bins):
        left, right = bins[b], bins[b+1]
        mask = (y_prob >= left) & (y_prob < right) if b < n_bins-1 else (y_prob >= left) & (y_prob <= right)
        if not np.any(mask):
            continue
        acc = y_true[mask].mean()
        conf = y_prob[mask].mean()
        ece += (mask.sum() / total) * abs(acc - conf)
    return float(ece)

def build_feature_df(X_base: pd.DataFrame, cols: List[str], W: np.ndarray, feature_type: str = "mul", top_k: int = 15):
    edges = pick_top_k_edges(W, cols, top_k=top_k)
    feats = {}
    for i, j, w_ij in edges:
        A_name = cols[j]; B_name = cols[i]
        if feature_type == "mul":
            name = f"{A_name}_mul_{B_name}"
            val = X_base[A_name] * X_base[B_name]
        elif feature_type == "mulw":
            name = f"{A_name}_mulw_{B_name}"
            val = w_ij * (X_base[A_name] * X_base[B_name])
        else:
            raise ValueError("feature_type must be 'mul' or 'mulw'")
        feats[name] = val.values
    return pd.DataFrame(feats, index=X_base.index)


# === LightGBM 교차검증 ===
def run_cv_lgbm(X: pd.DataFrame, y: pd.Series, n_splits=10, random_state=42, tag: str = "") -> Dict[str, float]:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    f1s, auprcs, aurocs, briers, eces = [], [], [], [], []

    X_np = X.values.astype(np.float32)
    y_np = y.values.astype(int)

    for fold, (tr_idx, te_idx) in enumerate(skf.split(X_np, y_np), start=1):
        if PRINT_PROGRESS and tag:
            print(f"[{tag}] fold {fold}/{n_splits} ...", flush=True)

        X_tr, X_te = X_np[tr_idx], X_np[te_idx]
        y_tr, y_te = y_np[tr_idx], y_np[te_idx]

        # 내부 검증(조기 종료)
        X_tr_sub, X_val, y_tr_sub, y_val = train_test_split(
            X_tr, y_tr, test_size=0.2, random_state=random_state, stratify=y_tr
        )

        clf = LGBMClassifier(**LGBM_PARAMS)
        clf.fit(
            X_tr_sub, y_tr_sub,
            eval_set=[(X_val, y_val)],
            eval_metric="auc",
            callbacks=[
                early_stopping(EARLY_STOPPING_ROUNDS),
                log_evaluation(-1)
            ]
        )

        y_proba = clf.predict_proba(X_te)[:, 1]
        y_pred  = (y_proba >= 0.5).astype(int)

        f1s.append(f1_score(y_te, y_pred, zero_division=0))
        try:
            aurocs.append(roc_auc_score(y_te, y_proba))
        except Exception:
            aurocs.append(np.nan)
        try:
            auprcs.append(average_precision_score(y_te, y_proba))
        except Exception:
            auprcs.append(np.nan)
        briers.append(brier_score_loss(y_te, y_proba))
        eces.append(expected_calibration_error(y_te, y_proba, n_bins=10))

    return {
        "f1": float(np.nanmean(f1s)),
        "AUPRC": float(np.nanmean(auprcs)),
        "AUROC": float(np.nanmean(aurocs)),
        "Brier": float(np.nanmean(briers)),
        "ECE": float(np.nanmean(eces)),
    }


# 3) 파생 피처 생성 (랜덤 W 기반)
Xf_m   = build_feature_df(X_base[cols_used], cols_used, W_used, feature_type="mul",  top_k=TOP_K_EDGES)
Xf_mw  = build_feature_df(X_base[cols_used], cols_used, W_used, feature_type="mulw", top_k=TOP_K_EDGES)
Xf_mmw = pd.concat([Xf_m, Xf_mw], axis=1)

# 4) 평가
res_O      = run_cv_lgbm(X_base, y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag="O(original)")
res_F_m    = run_cv_lgbm(Xf_m,  y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag="F(m)")
res_OF_m   = run_cv_lgbm(pd.concat([X_base, Xf_m], axis=1),  y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag="OF(m)")

res_F_mw   = run_cv_lgbm(Xf_mw, y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag="F(mw)")
res_OF_mw  = run_cv_lgbm(pd.concat([X_base, Xf_mw], axis=1), y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag="OF(mw)")

res_F_mmw  = run_cv_lgbm(Xf_mmw, y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag="F(m+mw)")
res_OF_mmw = run_cv_lgbm(pd.concat([X_base, Xf_mmw], axis=1), y, n_splits=N_SPLITS, random_state=RANDOM_STATE, tag="OF(m+mw)")

# 5) 결과 표
col_top = ["O", "F", "OF"]
col_metrics = ["f1", "AUPRC", "AUROC", "Brier", "ECE"]
multi_cols = pd.MultiIndex.from_product([col_top, col_metrics])
index_rows = ["original", "m", "mw", "m+mw"]

report = pd.DataFrame(index=index_rows, columns=multi_cols, dtype=float)

# original 행: O만
for met in col_metrics:
    report.loc["original", ("O", met)] = res_O[met]

# m / mw / m+mw 행
def put(row, res_F, res_OF):
    for met in col_metrics:
        report.loc[row, ("O", met)]  = res_O[met]
        report.loc[row, ("F", met)]  = res_F[met]
        report.loc[row, ("OF", met)] = res_OF[met]

put("m",    res_F_m,   res_OF_m)
put("mw",   res_F_mw,  res_OF_mw)
put("m+mw", res_F_mmw, res_OF_mmw)

report = report.astype(float).round(4)
print(report)


[O(original)] fold 1/10 ...
[LightGBM] [Info] Number of positive: 285, number of negative: 12588
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000370 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3030
[LightGBM] [Info] Number of data points in the train set: 12873, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Start training from score 0.000000
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[22]	valid_0's auc: 0.928258	valid_0's binary_logloss: 0.286746
[O(original)] fold 2/10 ...
[LightGBM] [Info] Number of positive: 286, number of negative: 12588
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000349 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3026
[LightGBM] [Info] Number of d

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Early stopping, best iteration is:
[106]	valid_0's auc: 0.920933	valid_0's binary_logloss: 0.0914904
[O(original)] fold 3/10 ...
[LightGBM] [Info] Number of positive: 286, number of negative: 12588
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000365 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3025
[LightGBM] [Info] Number of data points in the train set: 12874, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Training until validation scores don't improve for 50 rounds


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Early stopping, best iteration is:
[82]	valid_0's auc: 0.953858	valid_0's binary_logloss: 0.105681
[O(original)] fold 4/10 ...
[LightGBM] [Info] Number of positive: 286, number of negative: 12588
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000352 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3034
[LightGBM] [Info] Number of data points in the train set: 12874, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Training until validation scores don't improve for 50 rounds


...

c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Early stopping, best iteration is:
[119]	valid_0's auc: 0.922307	valid_0's binary_logloss: 0.0815163
[OF(m+mw)] fold 10/10 ...
[LightGBM] [Info] Number of positive: 285, number of negative: 12589
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001892 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 10683
[LightGBM] [Info] Number of data points in the train set: 12874, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
Training until validation scores don't improve for 50 rounds


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


Early stopping, best iteration is:
[91]	valid_0's auc: 0.922495	valid_0's binary_logloss: 0.0972147
               O                                      F                  \
              f1   AUPRC   AUROC  Brier     ECE      f1   AUPRC   AUROC   
original  0.3923  0.4337  0.9152  0.042  0.0952     NaN     NaN     NaN   
m         0.3923  0.4337  0.9152  0.042  0.0952  0.2869  0.2621  0.8719   
mw        0.3923  0.4337  0.9152  0.042  0.0952  0.2868  0.2674  0.8742   
m+mw      0.3923  0.4337  0.9152  0.042  0.0952  0.2833  0.2645  0.8744   

                              OF                                  
           Brier     ECE      f1   AUPRC   AUROC   Brier     ECE  
original     NaN     NaN     NaN     NaN     NaN     NaN     NaN  
m         0.0479  0.1034  0.4018  0.4472  0.9118  0.0467  0.1054  
mw        0.0462  0.0975  0.3970  0.4372  0.9089  0.0424  0.1074  
m+mw      0.0434  0.0858  0.4020  0.4665  0.9146  0.0298  0.0572  


c:\Users\User\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [ ]:
# report: MultiIndex 컬럼의 결과 DataFrame이 이미 존재한다고 가정
# format 옵션: "multi" | "flat" | "long"
save_format = "multi"  # 원하는 포맷으로 변경

if save_format == "multi":
    out_path = "report_ges_lightgbm_rand_edge_w.csv"
    report.to_csv(out_path, encoding="utf-8-sig")

elif save_format == "flat":
    out_path = "report_ges_lightgbm_rand_edge_w.csv"
    flat_cols = [f"{top}_{met}" for top, met in report.columns.to_list()]
    report_flat = report.copy()
    report_flat.columns = flat_cols
    report_flat.to_csv(out_path, encoding="utf-8-sig")

elif save_format == "long":
    out_path = "report_ges_lightgbm_rand_edge_w.csv"
    report_long = (
        report
        .rename_axis(index="row", columns=["set", "metric"])
        .stack(["set", "metric"])
        .to_frame("value")
        .reset_index()[["row", "set", "metric", "value"]]
    )
    report_long.to_csv(out_path, index=False, encoding="utf-8-sig")
else:
    raise ValueError("save_format must be one of: 'multi', 'flat', 'long'")

print(f"Saved: {out_path} (format={save_format})")


Saved: report_ges_lightgbm_rand_edge_w.csv (format=multi)
